<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       SInfo function in Vantage
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial'><b>Introduction</b></p>

<p style = 'font-size:20px;font-family:Arial'><b>SInfo</b></p>
<p style = 'font-size:16px;font-family:Arial'>When working with time series data, it is good to know the data characteristics before conducting any analysis. Knowing the characteristics is useful in the following several ways:</p>

<li style = 'font-size:16px;font-family:Arial'><code>Data quality:</code> Identify errors or inconsistencies in the data. If the data type is supposed to be numeric but there are non-numeric values, it can indicate a data quality issue.</li>
<li style = 'font-size:16px;font-family:Arial'><code>Understand data:</code> Know the interval and frequency of the time series to understand the underlying process generating the data. If the time series is hourly, daily, or weekly, it can indicate the level of granularity of the data and the potential patterns or trends.</li>
<li style = 'font-size:16px;font-family:Arial'><code>Analysis methods:</code> Determine the analysis methods suitable for different data types. If data has a seasonal pattern, a seasonal decomposition analysis can be appropriate. If data has a trend, a regression analysis can be useful.</li>
<li style = 'font-size:16px;font-family:Arial'><code>Outliers:</code> Know the minimum and maximum values of the data to identify outliers or unusual data points that may require further investigation.</li>
<li style = 'font-size:16px;font-family:Arial'><code>Visualization:</code> Design appropriate data visualization techniques to effectively communicate the insights to stakeholders.</li></p>

<p style = 'font-size:16px;font-family:Arial'>SInfo() returns one row for each series instance found in the target table. Each returned row provides the following information about the series:</p>
<li style = 'font-size:16px;font-family:Arial'>Index data type</li>
<li style = 'font-size:16px;font-family:Arial'>Starting index value</li>
<li style = 'font-size:16px;font-family:Arial'>Ending index value</li>
<li style = 'font-size:16px;font-family:Arial'>Number of series entries</li>
<li style = 'font-size:16px;font-family:Arial'>Indicator that the series is regular (discrete) or irregular</li>
<li style = 'font-size:16px;font-family:Arial'>Sample interval for regular series or average sample interval for irregular series</li>
<li style = 'font-size:16px;font-family:Arial'>Content type</li>
<li style = 'font-size:16px;font-family:Arial'>Minimum sample magnitude</li>
<li style = 'font-size:16px;font-family:Arial'>Maximum sample magnitude</li>
<li style = 'font-size:16px;font-family:Arial'>Average of magnitudes in the series</li>
<li style = 'font-size:16px;font-family:Arial'>Root-mean-square for each magnitude</li></p>


<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>1. Initiate a connection to Vantage</b>

<p style = 'font-size:16px;font-family:Arial'>In the section, we import the required libraries and set environment variables and environment paths (if required).

In [ ]:
from teradataml import (
    create_context,
    execute_sql,
    load_example_data,
    DataFrame,
    in_schema,
    TDSeries,
    SInfo,
    remove_context,
    copy_to_sql,
    db_drop_table
    )

from teradatasqlalchemy.types import *

from teradataml import to_numeric
# Modify the following to match the specific client environment settings
display.max_rows = 5

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>1.1 Connect to Vantage</b></p>
<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../../UseCases/startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=PP_SInfo.ipynb;' UPDATE FOR SESSION; ''')

<p style = 'font-size:16px;font-family:Arial'>Begin running steps with Shift + Enter keys. </p>

<hr style='height:1px;border:none;'>

<p style = 'font-size:18px;font-family:Arial'><b>1.2 Getting Data for This Demo</b></p>

<p style = 'font-size:16px;font-family:Arial'>Here, we will get the time series data which is available in the teradataml library and use the same to show the usage of the function.</p>

In [ ]:
load_example_data("uaf", ["ocean_buoys2"])

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>2. Data Exploration</b>
<p style = 'font-size:16px;font-family:Arial'>Create a "Virtual DataFrame" that points to the data set in Vantage. Check the shape of the dataframe as check the datatype of all the columns of the dataframe.</p>

In [ ]:
df = DataFrame.from_table("ocean_buoys2")
df

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>3. SInfo</b>
<p style = 'font-size:16px;font-family:Arial'>SInfo() displays a result set with metadata about the series. There is one output row for each series. In addition, each input payload produces the output variables for MIN, MAX, AVG and RMS. The output varies depending on the number of input payloads.</p>

<p style = 'font-size:16px;font-family:Arial'>Detailed help can be found by passing function name to built-in help function. </p>

In [ ]:
help(SInfo)

<p style = 'font-size:16px;font-family:Arial'>The first step is to convert the data into TDSeries, which is required for the input time series which are passed to the SInfo function.</p>

In [ ]:
data_series_df = TDSeries(data=df,
                              id=["ocean_name","buoyid"],
                              row_index="TD_TIMECODE",
                              row_index_style="TIMECODE",
                              payload_field="jsoncol.Measure.salinity",
                              payload_content="REAL")

In [ ]:
uaf_out = SInfo(data=data_series_df)

sinfo_df = uaf_out.result
sinfo_df

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>4. Cleanup</b>

<p style = 'font-size:18px;font-family:Arial'><b>Work Tables</b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up tables created above.</p>

In [ ]:
db_drop_table("ocean_buoys2")

In [ ]:
remove_context()

<hr style="height:1px;border:none;">

<p style = 'font-size:16px;font-family:Arial'><b>Links:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Teradataml Python reference: <a href = 'https://docs.teradata.com/search/all?query=Python+Package+User+Guide&content-lang=en-US'>here</a></li>
    <li>SInfo function reference: <a href = 'https://docs.teradata.com/search/all?query=SInfo&content-lang=en-US'>here</a></li>
    
</ul>

<footer style="padding-bottom:35px; border-bottom:3px solid #91A0Ab">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2025. All Rights Reserved
        </div>
    </div>
</footer>